In [1]:
import sys
sys.path.append('../src/')
import decode_mcd
import decode_mcd.multi_objective_problem as MOP
from decode_mcd import data_package
from decode_mcd import design_targets
from decode_mcd import counterfactuals_generator

In [14]:
import random

from pymoo.core.variable import Real

import numpy as np

x = np.random.random(100)
x = x.reshape(100, 1)
y = x * 100


def predict(_x):
    return _x * 100 + random.random()

def validity(_x):
    return _x % 0.1 < 0.05

data = data_package.DataPackage(features_dataset=x,
                           predictions_dataset=y,
                           query_x=x[0].reshape(1, 1),
                           design_targets=design_targets.DesignTargets([design_targets.ContinuousTarget(label=0,
                                                                          lower_bound=25,
                                                                          upper_bound=75)]),
                           datatypes=[Real(bounds=(0, 1))])

problem = MOP.MultiObjectiveProblem(data_package=data,
                                prediction_function=lambda design: predict(design),
                                constraint_functions=[validity])

generator = counterfactuals_generator.CounterfactualsGenerator(problem=problem,
                                     pop_size=10,
                                     initialize_from_dataset=False)

generator.generate(n_generations=10)
counterfactuals = generator.sample_with_dtai(num_samples=10, gower_weight=1,
                                             avg_gower_weight=1, cfc_weight=1,
                                             diversity_weight=50)
print(counterfactuals)

Initial population randomly initialized!
Training GA from 0 to 10 generations!
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        9 |      1 |  1.0000000000 |  2.0000000000 |             - |             -
     2 |       19 |      1 |  0.000000E+00 |  1.0000000000 |             - |             -
     3 |       29 |      1 |  0.000000E+00 |  0.7000000000 |  0.000000E+00 |             f
     4 |       39 |      2 |  0.000000E+00 |  0.3000000000 |  3.4844101382 |         ideal
     5 |       49 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     6 |       59 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |       69 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     8 |       79 |      1 |  0.000000E+00 |  0.000000E+00 |  0.0162290166 |         nadir
     9 |       89 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |   